# Course:  Convolutional Neural Networks for Image Classification

## Section-5
### Construct deep architectures for CNN models
#### What else?

**Description:**  
*Improve designed deep network by adding advanced features  
Interpret notation*

**File:** *advanced_features.ipynb*

### Algorithm:

**--> Step 1:** Open preprocessed dataset  
**--> Step 2:** Convert classes vectors to binary matrices  
**--> Step 3:** **Replace max pooling by advanced features**  
**--> Step 4:** Visualize built CNN models  
**--> Step 5:** Set up learning rate & epochs  
**--> Step 6:** Train built CNN models  
**--> Step 7:** Show and plot accuracies  
**--> Step 8:** Make a conclusion  


**Result:**  
- Chosen architecture for every preprocessed dataset  


## Importing libraries

In [ ]:
# Importing needed libraries
import matplotlib.pyplot as plt
import numpy as np
import h5py


from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, AvgPool2D
from keras.callbacks import LearningRateScheduler
from keras.utils import plot_model


## Setting up full path to preprocessed datasets

In [ ]:
# Full or absolute path to 'Section4' with preprocessed datasets
# (!) On Windows, the path should look like following:
# r'C:\Users\your_name\PycharmProjects\CNNCourse\Section4'
# or:
# 'C:\\Users\\your_name\\PycharmProjects\\CNNCourse\\Section4'
full_path_to_Section4 = \
    '/home/valentyn/PycharmProjects/CNNCourse/Section4'


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved custom dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 'custom' + '/' +
               'dataset_custom_rgb_255_mean_std.hdf5', 'r') as f:
    
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 2: Converting classes vectors to classes matrices

In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[4])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 5)
y_validation = to_categorical(y_validation, num_classes = 5)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[4])


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 3: Replacing max pooling by advanced features

### Notation

**C** - convolutional layer  
**P** - pooling  
**D** - dropout  
  
Examples:
* **8C5** - convolutional layer with 8 feature maps and kernels of spatial size 5x5  
* **8C2S2** - the same as above, but with stride 2 and kernels of spatial size 2x2  
* **P2** - pooling operation with 2x2 window and stride 2  
* **AP2** - average pooling operation with 2x2 window and stride 2  
*  **128** - fully connected layer (dense layer) with 128 neurons  
*  **D15** - 15% of dropout  
  
Definitions:
* **filters** (also called as kernels or cores) are trainable parameters  
* **weights** are values of filters that network learns during training  
* **strides** are steps by which window of filter size goes through the input  
* **padding** is a 0-valued frame used to process edges of the input  
* **Dropout** is a regularization technique that helps to prevent overfitting  
  
Some keywords values:
* **kernel_size=5** sets the filter size to be 5x5
* **strides=1** is a default value
* **padding='valid'** is a default value, meaning that output will be reduced: kernel_size - 1  
* **padding='same'** means that output will be of the same spatial size as input  
* **activation='relu'** sets ReLU (Rectified Linear Unit) as activation function  
  
Calculations of spatial size for feature maps after convolutional layer:  
* **height_output = 1 + (height_input + 2 * pad - kernel_size) / stride**
* **width_output = 1 + (width_input + 2 * pad - kernel_size) / stride**
  
Example without pad frame:
* **height_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
* **width_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
  
Example with pad frame:
* **height_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
* **width_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
  

In [ ]:
# Building 3 models
# RGB --> {64C5-P2-D20} --> {128C5-P2-D20} --> {256C5-P2-D20} --> {512C5-P2-D20} --> 2048-D20 --> 5
# RGB --> {64C5-64C2S2-D20} --> {128C5-128C2S2-D20} --> {256C5-256C2S2-D20} --> {512C5-512C2S2-D20} --> 2048-D20 --> 5
# RGB --> {64C5-AP2-D20} --> {128C5-AP2-D20} --> {256C5-AP2-D20} --> {512C5-AP2-D20} --> 2048-D20 --> 5


# Defining list to collect models in
model = []


# Building 1st model
# RGB --> {64C5-P2-D20} --> {128C5-P2-D20} --> {256C5-P2-D20} --> {512C5-P2-D20} --> 2048-D20 --> 5

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling pair
temp.add(Conv2D(64, kernel_size=5, padding='same', activation='relu', input_shape=(64, 64, 3)))
temp.add(MaxPool2D())
temp.add(Dropout(0.2))

# Adding second convolutional-pooling pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.2))

# Adding third convolutional-pooling pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.2))

# Adding fourth convolutional-pooling pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.2))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(2048, activation='relu'))
temp.add(Dropout(0.2))
temp.add(Dense(5, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Adding current model in the list
model.append(temp)



# Building 2nd model
# RGB --> {64C5-64C2S2-D20} --> {128C5-128C2S2-D20} --> {256C5-256C2S2-D20} --> {512C5-512C2S2-D20} --> 2048-D20 --> 5

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-convolutional pair
temp.add(Conv2D(64, kernel_size=5, padding='same', activation='relu', input_shape=(64, 64, 3)))
temp.add(Conv2D(64, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.2))

# Adding second convolutional-convolutional pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(128, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.2))

# Adding third convolutional-convolutional pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(256, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.2))

# Adding fourth convolutional-convolutional pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(512, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.2))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(2048, activation='relu'))
temp.add(Dropout(0.2))
temp.add(Dense(5, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Adding current model in the list
model.append(temp)



# Building 3rd model
# RGB --> {64C5-AP2-D20} --> {128C5-AP2-D20} --> {256C5-AP2-D20} --> {512C5-AP2-D20} --> 2048-D20 --> 5

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling(average) pair
temp.add(Conv2D(64, kernel_size=5, padding='same', activation='relu', input_shape=(64, 64, 3)))
temp.add(AvgPool2D())
temp.add(Dropout(0.2))

# Adding second convolutional-pooling(average) pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.2))

# Adding third convolutional-pooling(average) pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.2))

# Adding fourth convolutional-pooling(average) pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.2))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(2048, activation='relu'))
temp.add(Dropout(0.2))
temp.add(Dense(5, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Adding current model in the list
model.append(temp)



# Check point
print('3 models are compiled successfully')


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 4: Visualizing built CNN models

In [ ]:
# Plotting model's layers in form of flowchart
plot_model(model[1],
           to_file='model.png',
           show_shapes=True,
           show_layer_names=False,
           rankdir='TB',
           dpi=500)


In [ ]:
# Showing model's summary in form of table
model[2].summary()


In [ ]:
# Showing dropout rate
print(model[1].layers[2].rate)

# Showing strides for the 1st convolutional layer
print(model[1].layers[0].strides)

# Showing strides for the 2nd convolutional layer
print(model[1].layers[1].strides)


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 5: Setting up learning rate & epochs

In [ ]:
# Defining number of epochs
epochs = 20

# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs), verbose=1)

# Check point
print('Number of epochs and schedule for learning rate are set successfully')


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 6: Training built CNN models

In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Defining list to collect results in
h = []


# Training models in a loop
for i in range(3):
    # Current model
    temp = model[i].fit(x_train, y_train,
                        batch_size=50,
                        epochs=epochs,
                        validation_data=(x_validation, y_validation),
                        callbacks=[learning_rate],
                        verbose=1)
    
    # Adding results of current model in the list
    h.append(temp)


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 7: Showing and plotting accuracies

In [ ]:
# Accuracies of the models
for i in range(3):
    print('Model {0}: Training accuracy={1:.5f}, Validation accuracy={2:.5f}'.
                                                          format(i + 1,
                                                                 max(h[i].history['accuracy']),
                                                                 max(h[i].history['val_accuracy'])))


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Setting default size of the plot
plt.rcParams['figure.figsize'] = (12.0, 6.0)


# Plotting accuracies for every model
plt.plot(h[0].history['val_accuracy'], '-o')
plt.plot(h[1].history['val_accuracy'], '-o')
plt.plot(h[2].history['val_accuracy'], '-o')


# Setting limit along Y axis
plt.ylim(0.53, 0.72)


# Showing legend
plt.legend(['model_1', 'model_2', 'model_3'], loc='lower right', fontsize='xx-large')


# Giving name to axes
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)


# Giving name to the plot
plt.title('Models accuracies: Custom Dataset', fontsize=16)


# Showing the plot
plt.show()


In [ ]:
# Showing list of scheduled learning rate for every epoch
print(h[0].history['lr'])


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Plotting scheduled learning rate
plt.plot(h[0].history['lr'], '-mo')


# Showing the plot
plt.show()


### RGB custom dataset (255.0 ==> mean ==> std)

## Step 8: Making a conclusion

In [ ]:
# According to validation accuracy, the 3rd model has the highest value

# The choice for custom dataset is 3rd model
# RGB --> {64C5-AP2-D20} --> {128C5-AP2-D20} --> {256C5-AP2-D20} --> {512C5-AP2-D20} --> 2048-D20 --> 5
# GRAY input --> {64C5-AP2-D20} --> {128C5-AP2-D20} --> {256C5-AP2-D20} --> {512C5-AP2-D20} --> 2048-D20 --> 5

# RGB input: (64, 64, 3)
# GRAY input: (64, 64, 1)


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved CIFAR-10 dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 'cifar10' + '/' +
               'dataset_cifar10_rgb_255_mean_std.hdf5', 'r') as f:
    
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables    
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables 
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables 
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 2: Converting classes vectors to classes matrices

In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[4])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 10)
y_validation = to_categorical(y_validation, num_classes = 10)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[4])


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 3: Replacing max pooling by advanced features

### Notation

**C** - convolutional layer  
**P** - pooling  
**D** - dropout  
  
Examples:
* **8C5** - convolutional layer with 8 feature maps and kernels of spatial size 5x5  
* **8C2S2** - the same as above, but with stride 2 and kernels of spatial size 2x2  
* **P2** - pooling operation with 2x2 window and stride 2  
* **AP2** - average pooling operation with 2x2 window and stride 2  
*  **128** - fully connected layer (dense layer) with 128 neurons  
*  **D15** - 15% of dropout  
  
Definitions:
* **filters** (also called as kernels or cores) are trainable parameters  
* **weights** are values of filters that network learns during training  
* **strides** are steps by which window of filter size goes through the input  
* **padding** is a 0-valued frame used to process edges of the input  
* **Dropout** is a regularization technique that helps to prevent overfitting  
  
Some keywords values:
* **kernel_size=5** sets the filter size to be 5x5
* **strides=1** is a default value
* **padding='valid'** is a default value, meaning that output will be reduced: kernel_size - 1  
* **padding='same'** means that output will be of the same spatial size as input  
* **activation='relu'** sets ReLU (Rectified Linear Unit) as activation function  
  
Calculations of spatial size for feature maps after convolutional layer:  
* **height_output = 1 + (height_input + 2 * pad - kernel_size) / stride**
* **width_output = 1 + (width_input + 2 * pad - kernel_size) / stride**
  
Example without pad frame:
* **height_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
* **width_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
  
Example with pad frame:
* **height_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
* **width_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
  

In [ ]:
# Building 3 models
# RGB --> {128C5-P2-D40} --> {256C5-P2-D40} --> {512C5-P2-D40} --> 256-D40 --> 10
# RGB --> {128C5-128C2S2-D40} --> {256C5-256C2S2-D40} --> {512C5-512C2S2-D40} --> 256-D40 --> 10
# RGB --> {128C5-AP2-D40} --> {256C5-AP2-D40} --> {512C5-AP2-D40} --> 256-D40 --> 10


# Defining list to collect models in
model = []


# Building 1st model
# RGB --> {128C5-P2-D40} --> {256C5-P2-D40} --> {512C5-P2-D40} --> 256-D40 --> 10

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(32, 32, 3)))
temp.add(MaxPool2D())
temp.add(Dropout(0.4))

# Adding second convolutional-pooling pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.4))

# Adding third convolutional-pooling pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.4))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(256, activation='relu'))
temp.add(Dropout(0.4))
temp.add(Dense(10, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Building 2nd model
# RGB --> {128C5-128C2S2-D40} --> {256C5-256C2S2-D40} --> {512C5-512C2S2-D40} --> 256-D40 --> 10

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-convolutional pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(32, 32, 3)))
temp.add(Conv2D(128, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.4))

# Adding second convolutional-convolutional pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(256, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.4))

# Adding third convolutional-convolutional pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(512, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.4))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(256, activation='relu'))
temp.add(Dropout(0.4))
temp.add(Dense(10, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Building 3rd model
# RGB --> {128C5-AP2-D40} --> {256C5-AP2-D40} --> {512C5-AP2-D40} --> 256-D40 --> 10

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling(average) pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(32, 32, 3)))
temp.add(AvgPool2D())
temp.add(Dropout(0.4))

# Adding second convolutional-pooling(average) pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.4))

# Adding third convolutional-pooling(average) pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.4))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(256, activation='relu'))
temp.add(Dropout(0.4))
temp.add(Dense(10, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Check point
print('3 models are compiled successfully')


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 4: Visualizing built CNN models

In [ ]:
# Plotting model's layers in form of flowchart
plot_model(model[1],
           to_file='model.png',
           show_shapes=True,
           show_layer_names=False,
           rankdir='TB',
           dpi=500)


In [ ]:
# Showing model's summary in form of table
model[2].summary()


In [ ]:
# Showing dropout rate
print(model[1].layers[2].rate)

# Showing strides for the 1st convolutional layer
print(model[1].layers[0].strides)

# Showing strides for the 2nd convolutional layer
print(model[1].layers[1].strides)


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 5: Setting up learning rate & epochs

In [ ]:
# Defining number of epochs
epochs = 20

# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs), verbose=1)

# Check point
print('Number of epochs and schedule for learning rate are set successfully')


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 6: Training built CNN models

In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Defining list to collect results in
h = []


# Training models in a loop
for i in range(3):
    # Сurrent model
    temp = model[i].fit(x_train, y_train,
                        batch_size=50,
                        epochs=epochs,
                        validation_data=(x_validation, y_validation),
                        callbacks=[learning_rate],
                        verbose=1)
    
    # Adding results of current model in the list
    h.append(temp)


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 7: Showing and plotting accuracies

In [ ]:
# Accuracies of the models
for i in range(3):
    print('Model {0}: Training accuracy={1:.5f}, Validation accuracy={2:.5f}'.
                                                          format(i + 1,
                                                                 max(h[i].history['accuracy']),
                                                                 max(h[i].history['val_accuracy'])))


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Setting default size of the plot
plt.rcParams['figure.figsize'] = (12.0, 6.0)


# Plotting accuracies for every model
plt.plot(h[0].history['val_accuracy'], '-o')
plt.plot(h[1].history['val_accuracy'], '-o')
plt.plot(h[2].history['val_accuracy'], '-o')


# Setting limit along Y axis
plt.ylim(0.7, 0.84)


# Showing legend
plt.legend(['model_1', 'model_2', 'model_3'], loc='lower right', fontsize='xx-large')


# Giving name to axes
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)


# Giving name to the plot
plt.title('Models accuracies: CIFAR-10 dataset', fontsize=16)


# Showing the plot
plt.show()


In [ ]:
# Showing list of scheduled learning rate for every epoch
print(h[0].history['lr'])


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Plotting scheduled learning rate
plt.plot(h[0].history['lr'], '-mo')


# Showing the plot
plt.show()


### RGB CIFAR-10 dataset (255.0 ==> mean ==> std)

## Step 8: Making a conclusion

In [ ]:
# According to validation accuracy, the 1st model has the highest value

# The choice for CIFAR-10 dataset is 1st model
# RGB input --> {128C5-P2-D40} --> {256C5-P2-D40} --> {512C5-P2-D40} --> 256-D40 --> 10
# GRAY input --> {128C5-P2-D40} --> {256C5-P2-D40} --> {512C5-P2-D40} --> 256-D40 --> 10

# RGB input: (32, 32, 3)
# GRAY input: (32, 32, 1)


### GRAY MNIST dataset (255.0 ==> mean ==> std)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved MNIST dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 'mnist' + '/' +
               'dataset_mnist_gray_255_mean_std.hdf5', 'r') as f:
    
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables    
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables 
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables 
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


### GRAY MNIST dataset (255.0 ==> mean ==> std)

## Step 2: Converting classes vectors to classes matrices

In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[4])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 10)
y_validation = to_categorical(y_validation, num_classes = 10)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[4])


### GRAY MNIST dataset (255.0 ==> mean ==> std)

## Step 3: Replacing max pooling by advanced features

### Notation

**C** - convolutional layer  
**P** - pooling  
**D** - dropout  
  
Examples:
* **8C5** - convolutional layer with 8 feature maps and kernels of spatial size 5x5  
* **8C2S2** - the same as above, but with stride 2 and kernels of spatial size 2x2  
* **P2** - pooling operation with 2x2 window and stride 2  
* **AP2** - average pooling operation with 2x2 window and stride 2  
*  **128** - fully connected layer (dense layer) with 128 neurons  
*  **D15** - 15% of dropout  
  
Definitions:
* **filters** (also called as kernels or cores) are trainable parameters  
* **weights** are values of filters that network learns during training  
* **strides** are steps by which window of filter size goes through the input  
* **padding** is a 0-valued frame used to process edges of the input  
* **Dropout** is a regularization technique that helps to prevent overfitting  
  
Some keywords values:
* **kernel_size=5** sets the filter size to be 5x5
* **strides=1** is a default value
* **padding='valid'** is a default value, meaning that output will be reduced: kernel_size - 1  
* **padding='same'** means that output will be of the same spatial size as input  
* **activation='relu'** sets ReLU (Rectified Linear Unit) as activation function  
  
Calculations of spatial size for feature maps after convolutional layer:  
* **height_output = 1 + (height_input + 2 * pad - kernel_size) / stride**
* **width_output = 1 + (width_input + 2 * pad - kernel_size) / stride**
  
Example without pad frame:
* **height_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
* **width_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
  
Example with pad frame:
* **height_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
* **width_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
  

In [ ]:
# Building 3 models
# GRAY --> {128C5-P2-D30} --> {256C5-P2-D30} --> {512C5-P2-D30} --> 256-D30 --> 10
# GRAY --> {128C5-128C2S2-D30} --> {256C5-256C2S2-D30} --> {512C5-512C2S2-D30} --> 256-D30 --> 10
# GRAY --> {128C5-AP2-D30} --> {256C5-AP2-D30} --> {512C5-AP2-D30} --> 256-D30 --> 10


# Defining list to collect models in
model = []


# Building 1st model
# GRAY --> {128C5-P2-D30} --> {256C5-P2-D30} --> {512C5-P2-D30} --> 256-D30 --> 10

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(28, 28, 1)))
temp.add(MaxPool2D())
temp.add(Dropout(0.3))

# Adding second convolutional-pooling pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.3))

# Adding third convolutional-pooling pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.3))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(256, activation='relu'))
temp.add(Dropout(0.3))
temp.add(Dense(10, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Building 2nd model
# GRAY --> {128C5-128C2S2-D30} --> {256C5-256C2S2-D30} --> {512C5-512C2S2-D30} --> 256-D30 --> 10

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-convolutional pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(28, 28, 1)))
temp.add(Conv2D(128, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.3))

# Adding second convolutional-convolutional pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(256, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.3))

# Adding third convolutional-convolutional pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(512, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.3))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(256, activation='relu'))
temp.add(Dropout(0.3))
temp.add(Dense(10, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Building 3rd model
# GRAY --> {128C5-AP2-D30} --> {256C5-AP2-D30} --> {512C5-AP2-D30} --> 256-D30 --> 10

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling(average) pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(28, 28, 1)))
temp.add(AvgPool2D())
temp.add(Dropout(0.3))

# Adding second convolutional-pooling(average) pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.3))

# Adding third convolutional-pooling(average) pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.3))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(256, activation='relu'))
temp.add(Dropout(0.3))
temp.add(Dense(10, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Adding current model in the list
model.append(temp)



# Check point
print('3 models are compiled successfully')


### GRAY MNIST dataset (255.0 ==> mean ==> std)

## Step 4: Visualizing built CNN models

In [ ]:
# Plotting model's layers in form of flowchart
plot_model(model[1],
           to_file='model.png',
           show_shapes=True,
           show_layer_names=False,
           rankdir='TB',
           dpi=500)


In [ ]:
# Showing model's summary in form of table
model[2].summary()


In [ ]:
# Showing dropout rate
print(model[1].layers[2].rate)

# Showing strides for the 1st convolutional layer
print(model[1].layers[0].strides)

# Showing strides for the 2nd convolutional layer
print(model[1].layers[1].strides)


### GRAY MNIST dataset (255.0 ==> mean ==> std)

## Step 5: Setting up learning rate & epochs

In [ ]:
# Defining number of epochs
epochs = 20

# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs), verbose=1)

# Check point
print('Number of epochs and schedule for learning rate are set successfully')


### GRAY MNIST dataset (255.0 ==> mean ==> std)

## Step 6: Training built CNN models

In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Defining list to collect results in
h = []


# Training models in a loop
for i in range(3):
    # Сurrent model
    temp = model[i].fit(x_train, y_train,
                        batch_size=50,
                        epochs=epochs,
                        validation_data=(x_validation, y_validation),
                        callbacks=[learning_rate],
                        verbose=1)
    
    # Adding results of current model in the list
    h.append(temp)


### GRAY MNIST dataset (255.0 ==> mean ==> std)

## Step 7: Showing and plotting accuracies

In [ ]:
# Accuracies of the models
for i in range(3):
    print('Model {0}: Training accuracy={1:.5f}, Validation accuracy={2:.5f}'.
                                                          format(i + 1,
                                                                 max(h[i].history['accuracy']),
                                                                 max(h[i].history['val_accuracy'])))


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Setting default size of the plot
plt.rcParams['figure.figsize'] = (12.0, 6.0)


# Plotting accuracies for every model
plt.plot(h[0].history['val_accuracy'], '-o')
plt.plot(h[1].history['val_accuracy'], '-o')
plt.plot(h[2].history['val_accuracy'], '-o')


# Setting limit along Y axis
plt.ylim(0.987, 0.9951)


# Showing legend
plt.legend(['model_1', 'model_2', 'model_3'], loc='lower right', fontsize='xx-large')


# Giving name to axes
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)


# Giving name to the plot
plt.title('Models accuracies: MNIST dataset', fontsize=16)


# Showing the plot
plt.show()


In [ ]:
# Showing list of scheduled learning rate for every epoch
print(h[0].history['lr'])


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Plotting scheduled learning rate
plt.plot(h[0].history['lr'], '-mo')


# Showing the plot
plt.show()


### MNIST dataset (255.0 ==> mean ==> std)

## Step 8: Making a conclusion

In [ ]:
# According to validation accuracy, the 2nd model has the highest value

# The choice for MNIST dataset is 2nd model
# GRAY input --> {128C5-128C2S2-D30} --> {256C5-256C2S2-D30} --> {512C5-512C2S2-D30} --> 256-D30 --> 10

# GRAY input: (28, 28, 1)


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved Traffic Signs dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 'ts' + '/' +
               'dataset_ts_rgb_255_mean_std.hdf5', 'r') as f:
    
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables    
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables 
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables 
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 2: Converting classes vectors to classes matrices

In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[4])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 43)
y_validation = to_categorical(y_validation, num_classes = 43)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[4])


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 3: Replacing max pooling by advanced features

### Notation

**C** - convolutional layer  
**P** - pooling  
**D** - dropout  
  
Examples:
* **8C5** - convolutional layer with 8 feature maps and kernels of spatial size 5x5  
* **8C2S2** - the same as above, but with stride 2 and kernels of spatial size 2x2  
* **P2** - pooling operation with 2x2 window and stride 2  
* **AP2** - average pooling operation with 2x2 window and stride 2  
*  **128** - fully connected layer (dense layer) with 128 neurons  
*  **D15** - 15% of dropout  
  
Definitions:
* **filters** (also called as kernels or cores) are trainable parameters  
* **weights** are values of filters that network learns during training  
* **strides** are steps by which window of filter size goes through the input  
* **padding** is a 0-valued frame used to process edges of the input  
* **Dropout** is a regularization technique that helps to prevent overfitting  
  
Some keywords values:
* **kernel_size=5** sets the filter size to be 5x5
* **strides=1** is a default value
* **padding='valid'** is a default value, meaning that output will be reduced: kernel_size - 1  
* **padding='same'** means that output will be of the same spatial size as input  
* **activation='relu'** sets ReLU (Rectified Linear Unit) as activation function  
  
Calculations of spatial size for feature maps after convolutional layer:  
* **height_output = 1 + (height_input + 2 * pad - kernel_size) / stride**
* **width_output = 1 + (width_input + 2 * pad - kernel_size) / stride**
  
Example without pad frame:
* **height_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
* **width_output = 1 + (64 + 2 * 0 - 5) / 1 = 60**
  
Example with pad frame:
* **height_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
* **width_output = 1 + (64 + 2 * 2 - 5) / 1 = 64**
  

In [ ]:
# Building 3 models
# RGB --> {128C5-P2-D30} --> {256C5-P2-D30} --> {512C5-P2-D30} --> {1024C3-P2-D30} --> 2048-D30 --> 43
# RGB --> {128C5-128C2S2-D30} --> {256C5-256C2S2-D30} --> {512C5-512C2S2-D30} --> {1024C3-1024C2S2-D30} --> 2048-D30 --> 43
# RGB --> {128C5-AP2-D30} --> {256C5-AP2-D30} --> {512C5-AP2-D30} --> {1024C3-AP2-D30} --> 2048-D30 --> 43


# Defining list to collect models in
model = []


# Building 1st model
# RGB --> {128C5-P2-D30} --> {256C5-P2-D30} --> {512C5-P2-D30} --> {1024C3-P2-D30} --> 2048-D30 --> 43

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(48, 48, 3)))
temp.add(MaxPool2D())
temp.add(Dropout(0.3))

# Adding second convolutional-pooling pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.3))

# Adding third convolutional-pooling pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.3))

# Adding fourth convolutional-pooling pair
temp.add(Conv2D(1024, kernel_size=3, padding='same', activation='relu'))
temp.add(MaxPool2D())
temp.add(Dropout(0.3))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(2048, activation='relu'))
temp.add(Dropout(0.3))
temp.add(Dense(43, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Building 2nd model
# RGB --> {128C5-128C2S2-D30} --> {256C5-256C2S2-D30} --> {512C5-512C2S2-D30} --> {1024C3-1024C2S2-D30} --> 2048-D30 --> 43

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-convolutional pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(48, 48, 3)))
temp.add(Conv2D(128, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.3))

# Adding second convolutional-convolutional pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(256, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.3))

# Adding third convolutional-convolutional pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(Conv2D(512, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.3))

# Adding fourth convolutional-convolutional pair
temp.add(Conv2D(1024, kernel_size=3, padding='same', activation='relu'))
temp.add(Conv2D(1024, kernel_size=2, padding='same', activation='relu', strides=2))
temp.add(Dropout(0.3))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(2048, activation='relu'))
temp.add(Dropout(0.3))
temp.add(Dense(43, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Building 3rd model
# RGB --> {128C5-AP2-D30} --> {256C5-AP2-D30} --> {512C5-AP2-D30} --> {1024C3-AP2-D30} --> 2048-D30 --> 43

# Initializing model to be as linear stack of layers
temp = Sequential()

# Adding first convolutional-pooling(average) pair
temp.add(Conv2D(128, kernel_size=5, padding='same', activation='relu', input_shape=(48, 48, 3)))
temp.add(AvgPool2D())
temp.add(Dropout(0.3))

# Adding second convolutional-pooling(average) pair
temp.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.3))

# Adding third convolutional-pooling(average) pair
temp.add(Conv2D(512, kernel_size=5, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.3))

# Adding fourth convolutional-pooling(average) pair
temp.add(Conv2D(1024, kernel_size=3, padding='same', activation='relu'))
temp.add(AvgPool2D())
temp.add(Dropout(0.3))

# Adding fully connected layers
temp.add(Flatten())
temp.add(Dense(2048, activation='relu'))
temp.add(Dropout(0.3))
temp.add(Dense(43, activation='softmax'))

# Compiling created model
temp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
# Adding current model in the list
model.append(temp)



# Check point
print('3 models are compiled successfully')


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 4: Visualizing built CNN models

In [ ]:
# Plotting model's layers in form of flowchart
plot_model(model[1],
           to_file='model.png',
           show_shapes=True,
           show_layer_names=False,
           rankdir='TB',
           dpi=500)


In [ ]:
# Showing model's summary in form of table
model[2].summary()


In [ ]:
# Showing dropout rate
print(model[1].layers[2].rate)

# Showing strides for the 1st convolutional layer
print(model[1].layers[0].strides)

# Showing strides for the 2nd convolutional layer
print(model[1].layers[1].strides)


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 5: Setting up learning rate & epochs

In [ ]:
# Defining number of epochs
epochs = 20

# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs), verbose=1)

# Check point
print('Number of epochs and schedule for learning rate are set successfully')


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 6: Training built CNN models

In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Defining list to collect results in
h = []


# Training models in a loop
for i in range(3):
    # Сurrent model
    temp = model[i].fit(x_train, y_train,
                        batch_size=50,
                        epochs=epochs,
                        validation_data=(x_validation, y_validation),
                        callbacks=[learning_rate],
                        verbose=1)
    
    # Adding results of current model in the list
    h.append(temp)


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 7: Showing and plotting accuracies

In [ ]:
# Accuracies of the models
for i in range(3):
    print('Model {0}: Training accuracy={1:.5f}, Validation accuracy={2:.5f}'.
                                                         format(i + 1,
                                                                max(h[i].history['accuracy']),
                                                                max(h[i].history['val_accuracy'])))


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Setting default size of the plot
plt.rcParams['figure.figsize'] = (12.0, 6.0)


# Plotting accuracies for every model
plt.plot(h[0].history['val_accuracy'], '-o')
plt.plot(h[1].history['val_accuracy'], '-o')
plt.plot(h[2].history['val_accuracy'], '-o')


# Setting limit along Y axis
plt.ylim(0.988, 0.999)


# Showing legend
plt.legend(['model_1', 'model_2', 'model_3'], loc='lower right', fontsize='xx-large')


# Giving name to axes
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)


# Giving name to the plot
plt.title('Models accuracies: Traffic Signs dataset', fontsize=16)


# Showing the plot
plt.show()


In [ ]:
# Showing list of scheduled learning rate for every epoch
print(h[0].history['lr'])


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Plotting scheduled learning rate
plt.plot(h[0].history['lr'], '-mo')


# Showing the plot
plt.show()


### RGB Traffic Signs dataset (255.0 ==> mean ==> std)

## Step 8: Making a conclusion

In [ ]:
# According to validation accuracy, the 1st model has the highest value

# The choice for Traffic Signs dataset is 1st model
# RGB input --> {128C5-P2-D30} --> {256C5-P2-D30} --> {512C5-P2-D30} --> {1024C3-P2-D30} --> 2048-D30 --> 43
# GRAY input --> {128C5-P2-D30} --> {256C5-P2-D30} --> {512C5-P2-D30} --> {1024C3-P2-D30} --> 2048-D30 --> 43

# RGB input: (48, 48, 3)
# GRAY input: (48, 48, 1)


### Some comments

To get more details for usage of 'Max Pooling' class:  
**print(help(MaxPool2D))**  
  
More details and examples are here:  
https://keras.io/api/layers/pooling_layers/max_pooling2d/


To get more details for usage of 'Convolution' class:  
**print(help(Conv2D))**  
  
More details and examples are here:  
https://keras.io/api/layers/convolution_layers/convolution2d/


To get more details for usage of 'Average Pooling' class:  
**print(help(AvgPool2D))**  
  
More details and examples are here:  
https://keras.io/api/layers/pooling_layers/average_pooling2d/


To get more details for usage of 'Dropout' class:  
**print(help(Dropout))**  
  
More details and examples are here:  
https://keras.io/api/layers/regularization_layers/dropout/


To get more details for usage of 'Sequential' class:  
**print(help(Sequential))**  
  
More details and examples are here:  
https://keras.io/api/models/sequential/


To get more details for usage of function 'to_categorical':  
**print(help(to_categorical))**  

More details and examples are here:  
https://keras.io/api/utils/python_utils/#to_categorical-function 


To get more details for usage of function 'plot_model':  
**print(help(plot_model))**  

More details and examples are here:  
https://keras.io/api/utils/model_plotting_utils/#plot_model-function  


To get more details for usage of function 'plt.plot':  
**print(help(plt.plot))**  

More details and examples are here:  
https://matplotlib.org/3.1.3/api/_as_gen/matplotlib.pyplot.plot.html


In [ ]:
print(help(MaxPool2D))

In [ ]:
print(help(Conv2D))

In [ ]:
print(help(AvgPool2D))

In [ ]:
print(help(Dropout))

In [ ]:
print(help(Sequential))

In [ ]:
print(help(to_categorical))

In [ ]:
print(help(plot_model))

In [ ]:
print(help(plt.plot))

In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''dnn PoolForward launch failed [Op:MaxPool] name: max_pooling2d/MaxPool/'''
# In this case, switch to environment 'cnncpu', that is without GPU, and run again


# Importing needed libraries
import numpy as np
import tensorflow as tf


# Defining array with random values
x_train = np.random.randint(50, size=(4, 4)).astype(np.float32)

# Reshaping to get following: (batch size, rows, columns, channels)
x_train_input = x_train.reshape(1, 4, 4, 1)

# Showing initial array
print(x_train)
print()



# Initializing MaxPool2D layer
layer = tf.keras.layers.MaxPool2D()

# Passing array to the layer
output = layer(x_train_input)

# Showing array after MaxPool2D layer
print('MaxPool2D:', output.numpy().flatten())
print()



# Initializing AvgPool2D layer
layer = tf.keras.layers.AvgPool2D()

# Passing array to the layer
output = layer(x_train_input)

# Showing array after AvgPool2D layer
print('AvgPool2D:', output.numpy().flatten())
print()



# Initializing Conv2D layer
layer = tf.keras.layers.Conv2D(1, kernel_size=2, strides=2, kernel_initializer='ones')

# Passing array to the layer
output = layer(x_train_input)

# Showing array after Conv2D layer
print('Conv2D with stride 2:', output.numpy().flatten())
print()

# Showing kernels values
print('Kernels values:', layer.weights[0].numpy().flatten())
print()
